In [ ]:
import sys
import os
import tensorflow as tf

# Add the parent directory of the current notebook to sys.path
sys.path.append(os.path.abspath(".."))

import config

In [ ]:
class NestedImageResizer:
    def __init__(self, input_dir, output_dir):
        """
        Initialize the image resizer using global config parameters.
        
        Args:
            input_dir (str): Directory containing the 'fake' and 'real' subdirectories.
            output_dir (str): Directory to save the resized images.
        """
        self.input_dir = input_dir
        self.output_dir = output_dir

        # Directories for each class
        self.fake_dir = os.path.join(input_dir, "fake")
        self.real_dir = os.path.join(input_dir, "real")

        # Create output subdirectories for each class if they don't exist.
        os.makedirs(os.path.join(output_dir, "fake"), exist_ok=True)
        os.makedirs(os.path.join(output_dir, "real"), exist_ok=True)

    def _list_images(self, directory):
        """
        Recursively list all image files in a directory.
        
        Args:
            directory (str): Root directory to search for images.
        
        Returns:
            list: Full paths to image files.
        """
        image_paths = []
        for root, _, files in os.walk(directory):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    image_paths.append(os.path.join(root, file))
        return image_paths

    def _parse_and_resize_image(self, filename):
        """
        Read and resize an image from the given filename.
        
        Args:
            filename (str): Path to the image file.
        
        Returns:
            tf.Tensor: Resized image tensor in uint8.
        """
        # Read and decode the image
        image_raw = tf.io.read_file(filename)
        image = tf.image.decode_image(image_raw, channels=3)

        # Resize the image to IMG_SIZE (global configuration)
        image = tf.image.resize(image, config.IMG_SIZE)

        # tf.image.resize returns float32 values, so clip and cast back to uint8
        image = tf.clip_by_value(image, 0, 255)
        image = tf.cast(image, tf.uint8)
        return image

    def _save_resized_image(self, image, output_path):
        """
        Encode the image in JPEG format and write it to disk.
        
        Args:
            image (tf.Tensor): Image tensor to save.
            output_path (str): Path where the resized image will be stored.
        """
        encoded_image = tf.io.encode_jpeg(image)
        tf.io.write_file(output_path, encoded_image)

    def resize_and_save_images(self, max_images_per_class=None):
        """
        Resize images from both 'fake' and 'real' classes and save them to the output directory.
        
        Args:
            max_images_per_class (int, optional): Optionally limit the number of images processed per class.
        """
        # List images in both class directories
        fake_images = self._list_images(self.fake_dir)
        real_images = self._list_images(self.real_dir)

        if max_images_per_class:
            fake_images = fake_images[:max_images_per_class]
            real_images = real_images[:max_images_per_class]

        print(
            f"Processing {len(fake_images)} fake images and {len(real_images)} real images.")

        # Process fake images
        for file_path in fake_images:
            try:
                image = self._parse_and_resize_image(file_path)
                file_name = os.path.basename(file_path)
                output_path = os.path.join(self.output_dir, "fake", file_name)
                self._save_resized_image(image, output_path)
                print(f"Saved resized fake image to {output_path}")
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

        # Process real images
        for file_path in real_images:
            try:
                image = self._parse_and_resize_image(file_path)
                file_name = os.path.basename(file_path)
                output_path = os.path.join(self.output_dir, "real", file_name)
                self._save_resized_image(image, output_path)
                print(f"Saved resized real image to {output_path}")
            except Exception as e:
                print(f"Error processing {file_path}: {e}")


# Example usage:
if __name__ == "__main__":
    resizer = NestedImageResizer(
        input_dir=config.DATA_DIR, output_dir=config.OUTPUT_DIR)
    resizer.resize_and_save_images()